In [ ]:
%load_ext autoreload
%load_ext tensorboard

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

! pip install transformers datasets pytorch-lightning nltk
# Use widgets to add a form that turns off/on dev
# Use nbdev to generate notebooks
# import ipywidgets as widgets 
import os
os.chdir("/content/drive/MyDrive/ldh")

import nltk
nltk.download('punkt')


# import os
# os.chdir("/Users/danielpham/Google Drive/ldh")

Mounted at /content/drive
     |████████████████████████████████| 1.8MB 5.7MB/s 
     |████████████████████████████████| 163kB 26.8MB/s 
     |████████████████████████████████| 696kB 21.4MB/s 
     |████████████████████████████████| 890kB 33.2MB/s 
     |████████████████████████████████| 3.2MB 37.4MB/s 
     |████████████████████████████████| 20.7MB 1.3MB/s 
     |████████████████████████████████| 245kB 46.9MB/s 
     |████████████████████████████████| 276kB 57.7MB/s 
     |████████████████████████████████| 102kB 13.0MB/s 
     |████████████████████████████████| 829kB 50.6MB/s 
     |████████████████████████████████| 1.3MB 56.0MB/s 
     |████████████████████████████████| 296kB 58.2MB/s 
     |████████████████████████████████| 143kB 59.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=6502a5f0ab336b99e53c43be3208faf44cbdae11e60c4483168c7368dc2ea7f0
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5

True

## Data Preparation
- Data is loaded through LDHDataModule. 
- Returned 

In [7]:
%autoreload
from src.LDHDataModule import LDHDataModule
from src.LightningReapp import LightningReapp


ldhdata = LDHDataModule(batch_size=16)

Loading cached processed dataset at /content/drive/MyDrive/ldh/output/training/obj/cache-093d0fd442ef7738.arrow


Training data loaded from disk.
Encoding Train Data:
Evaluation data loaded from disk.
Encoding Test Data:


In [ ]:
# %tensorboard --logdir save_dir
import torch
import pytorch_lightning as lit
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

from pathlib import Path

save_dir = Path.cwd() / 'output'
logger = lit.loggers.TensorBoardLogger(save_dir=save_dir)
trainer = lit.Trainer(
    logger=logger,
    max_epochs=50,
    #fast_dev_run=1,
    gradient_clip_val=1.0,
    progress_bar_refresh_rate=30,
    gpus=1 if torch.cuda.is_available() else 0,
    terminate_on_nan=True,
    weights_summary="top",
    weights_save_path=save_dir,
    callbacks=[
      EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=True),
      ModelCheckpoint(
        monitor="val_loss",
        dirpath=save_dir,
        filename='reappraisal-model-{epoch_02d}-{val_loss:.2f}-gpu1',
        save_top_k=3,
        mode='min'
      )
    ]
  )

# Fit model
system = LightningReapp()
trainer.tune(system, ldhdata.train_dataloader())
# trainer.setup_trainer(system) #Sanity check 
trainer.fit(system, ldhdata.train_dataloader(), val_dataloaders=ldhdata.val_dataloader())
results = trainer.test(system, ldhdata.test_dataloader())

#5-fold CV: test on a reduced set of data
# for i in range(5):
#   ldhdata.current_split = i
#   system = LightningReapp()
#   print(f"=== Running Split {i} ===")
#   trainer.fit(system, ldhdata)

EarlyStopping mode set to min for monitoring val_loss.
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: Checkpoint directory /content/drive/MyDrive/ldh/output exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:50: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)

  | Name       | Type             | Params
------------------------------------------------
0 | model      | DistilBertModel  | 66.4 M
1 | avg        | AvgPool1d        | 0     
2 | classifier | Sequential       | 629 K 
3 | loss       | MeanSquaredError | 0     
------------------------------------------------
629 K     Trainable params
66.4 M    Non-trainable params
67.0 M    Total params


/usr/local/lib/python3.6/dist-packages/datasets/arrow_dataset.py:851: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.tensor(x, **format_kwargs)


AttributeError: ignored

# Test Dataset

In [9]:
result = trainer.test(system, ldhdata.test_dataloader())


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'predict':                                              response    output
0   There is a boy in a red shirt and jeans with w...  2.944729
1       There doesn't seem to be too much else to it.  2.600995
2                                       Can see ribs.  2.553155
3                                   Arms around head.  2.465945
4                                     curious though.  2.822324
5                          He is not wearing a shirt.  2.479888
6   This looks like he is in trouble or he is mad ...  2.897738
7                 Not highly complex, just kinda a w.  2.660939
8                             Very gloomy background.  2.793747
9                              Looking into distance.  3.179974
10                              His ribs are visible.  2.631243
11                                 He has his arms c.  2.629673}
-----------------------------------------------

In [15]:
test_dl = ldhdata.test_dataloader()

In [86]:
results = []
for batch in test_dl:
  out = system(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
  results.append((batch['addcode'], batch['response'], out.sum(dim=1).cpu()))

In [122]:
# results = pd.Series(results).map(lambda row: (row[0], row[1], row[2].detach().numpy()))
results_df = pd.DataFrame.from_records(results)

# .apply(
#     pd.DataFrame.from_records, axis=1
#   ).apply(
#     pd.DataFrame.transpose
#   ).dtypes

In [129]:
results_blown = results_df.apply([pd.DataFrame.from_records, pd.DataFrame.transpose], axis=1)

#pd.concat(results_df.apply([pd.DataFrame.from_records, pd.DataFrame.transpose]), axis=1)

In [150]:
new_df =results_blown.reset_index(drop=True)

In [163]:
new_df.columns = ['addcode', 'response', 'score']
new_df['score'] = new_df['score'].transform(torch.Tensor.detach).apply(torch.Tensor.numpy)

In [168]:
new_df.to_csv(save_dir / 'obj-predictions.csv')